# Testing out Google APis

I want to see how Google APIs work and how i can combine them together. 

Specifically I want to check out:

- Places API 
- StreetView API
 

I already got an API key and set it in my env file.

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

Call the places API in a radius around Bologna:

In [ ]:
import os
import requests
import json

api_key = os.getenv("GOOGLE_API_KEY")

# Example request to Places API - Text Search restricted to Bologna
query = "Giardini Margherita"
url = "https://places.googleapis.com/v1/places:searchText"

headers = {
    "Content-Type": "application/json",
    "X-Goog-Api-Key": api_key,
    'X-Goog-FieldMask' : '*'   # many fields available, use '*' to get all
}

#useful: location contains lat and lon

# Bologna coordinates
bologna_lat = 44.4949
bologna_lng = 11.3426
radius_meters = 5000  # 5km radius

payload = {
    "textQuery": query,
    "locationBias": {
        "circle": {
            "center": {
                "latitude": bologna_lat,
                "longitude": bologna_lng
            },
            "radius": radius_meters
        }
    },
    "maxResultCount": 5  # Limit to 5 results
}

response = requests.post(url, headers=headers, json=payload)
data = response.json()

print("Status Code:", response.status_code)
print(f"\nSearching for '{query}' in Bologna (radius: {radius_meters}m)")
print(f"\nNumber of results: {len(data.get('places', []))}")
print("\nResponse:")
print(json.dumps(data, indent=2))


Status Code: 200

Searching for 'Giardini Margherita' in Bologna (radius: 5000m)

Number of results: 1

Response:
{
  "places": [
    {
      "name": "places/ChIJLwKOhMXUf0cRiLrCD6oTfS0",
      "id": "ChIJLwKOhMXUf0cRiLrCD6oTfS0",
      "types": [
        "hiking_area",
        "tourist_attraction",
        "sports_activity_location",
        "park",
        "point_of_interest",
        "establishment"
      ],
      "nationalPhoneNumber": "051 203040",
      "internationalPhoneNumber": "+39 051 203040",
      "formattedAddress": "Viale Giovanni Gozzadini, 40136 Bologna BO, Italy",
      "addressComponents": [
        {
          "longText": "Viale Giovanni Gozzadini",
          "shortText": "Viale Giovanni Gozzadini",
          "types": [
            "route"
          ],
          "languageCode": "it"
        },
        {
          "longText": "Bologna",
          "shortText": "Bologna",
          "types": [
            "locality",
            "political"
          ],
          "langu

Return only needed fields: for example location, which contains lat & lon

In [7]:
import os
import requests
import json

api_key = os.getenv("GOOGLE_API_KEY")

# Example request to Places API - Text Search restricted to Bologna
query = "Giardini Margherita"
url = "https://places.googleapis.com/v1/places:searchText"

headers = {
    "Content-Type": "application/json",
    "X-Goog-Api-Key": api_key,
    'X-Goog-FieldMask' : 'places'   # many fields available, use '*' to get all
}

#useful: location contains lat and lon

# Bologna coordinates
bologna_lat = 44.4949
bologna_lng = 11.3426
radius_meters = 5000  # 5km radius

payload = {
    "textQuery": query,
    "locationBias": {
        "circle": {
            "center": {
                "latitude": bologna_lat,
                "longitude": bologna_lng
            },
            "radius": radius_meters
        }
    },
    "maxResultCount": 5  # Limit to 5 results
}

response = requests.post(url, headers=headers, json=payload)
data = response.json()

print("Status Code:", response.status_code)
print(f"\nSearching for '{query}' in Bologna (radius: {radius_meters}m)")
print(f"\nNumber of results: {len(data.get('places', []))}")
print("\nResponse:")
print(json.dumps(data, indent=2))
print("="*60)
print(data.get('places', [])[0].get('location', {}))


Status Code: 200

Searching for 'Giardini Margherita' in Bologna (radius: 5000m)

Number of results: 1

Response:
{
  "places": [
    {
      "name": "places/ChIJLwKOhMXUf0cRiLrCD6oTfS0",
      "id": "ChIJLwKOhMXUf0cRiLrCD6oTfS0",
      "types": [
        "hiking_area",
        "tourist_attraction",
        "park",
        "sports_activity_location",
        "point_of_interest",
        "establishment"
      ],
      "nationalPhoneNumber": "051 203040",
      "internationalPhoneNumber": "+39 051 203040",
      "formattedAddress": "Viale Giovanni Gozzadini, 40136 Bologna BO, Italy",
      "addressComponents": [
        {
          "longText": "Viale Giovanni Gozzadini",
          "shortText": "Viale Giovanni Gozzadini",
          "types": [
            "route"
          ],
          "languageCode": "it"
        },
        {
          "longText": "Bologna",
          "shortText": "Bologna",
          "types": [
            "locality",
            "political"
          ],
          "langu

In [9]:
import os
import requests

api_key = os.getenv("GOOGLE_API_KEY")

# Street View Static API URL
url = "https://maps.googleapis.com/maps/api/streetview"

# Use Piazza Maggiore coordinates (city center - has Street View coverage)
lat = 44.4949
lng = 11.3426

params = {
    "location": f"{lat},{lng}",  # Required: lat,lng coordinates
    "size": "640x640",            # Required: image size (max 640x640)
    "heading": 0,                 # Optional: camera direction in degrees (0-360)
    "pitch": 0,                   # Optional: vertical angle (-90 to 90, 0 = level)
    "fov": 90,                    # Optional: zoom level (10-120, higher = zoomed in)
    "key": api_key
}

response = requests.get(url, params=params)

print("Status Code:", response.status_code)
if response.status_code == 200:
    # Save the image
    with open("street_view_test.jpg", "wb") as f:
        f.write(response.content)
    print("Image saved as 'street_view_test.jpg'")
else:
    print(f"Error: {response.text}")

Status Code: 200
Image saved as 'street_view_test.jpg'
